In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset
import time
from tqdm import tqdm

In [2]:
# Configurations
device = "cpu"
learning_rate = 4.0
batch_size = 16
num_epochs = 5
marker = '.'
ngram_level = 4
hidden_dim = 20
embedding_dim = 20

In [3]:
# Create set & mappings
class CharacterDataset(Dataset):
    def __init__(self, words, characters):
        self.words = words
        self.characters = characters
        # Mapping characters to indices and vice versa
        self.char_to_idx = {ch: i for i, ch in enumerate(characters)}
        self.idx_to_char = {i: s for s, i in self.char_to_idx.items()}

    def __len__(self):
        return len(self.words)

    def contains(self, word):
        return word in self.words

    def get_vocab_size(self):
        return len(self.characters)

    def encode(self, word):
        # Convert a word to a list of character indices
        return [self.char_to_idx[c] for c in word]

    def decode(self, tokens):
        # Convert a list of character indices to a word
        return ''.join(self.idx_to_char[i] for i in tokens)

    def __getitem__(self, idx):
        word = self.words[idx]
        return self.encode(word)

In [4]:
#load data and create datasets

def load_datasets(window, input_file='data/names.txt'):
    with open(input_file, 'r') as f:
        data = f.read()
    words = data.splitlines()
    words = [w.strip() for w in words]
    words = [w for w in words if w]

    characters = sorted(list(set(''.join(words))))
    characters = [marker] + characters

    # Add marker characters to the beginning and end of each word
    words = [''.join((window - 1) * [marker]) + word + ''.join((window - 1) * [marker]) for word in words]

    print(f"The number of examples in the dataset: {len(words)}")
    print(f"The number of unique characters in the vocabulary: {len(characters)}")
    print(f"The vocabulary we have is: {''.join(characters)}")

    # Split the dataset into training, validation, and test sets
    out_of_sample_set_size = min(2000, int(len(words) * 0.1))
    test_set_size = 1500

    rp = torch.randperm(len(words)).tolist()

    train_words = [words[i] for i in rp[:-out_of_sample_set_size]]
    validation_words = [words[i] for i in rp[-out_of_sample_set_size:-test_set_size]]
    test_words = [words[i] for i in rp[-test_set_size:]]

    print(f"We've split up the dataset into {len(train_words)}, {len(validation_words)}, {len(test_words)} training, validation, and test examples")

    train_dataset = CharacterDataset(train_words, characters)
    validation_dataset = CharacterDataset(validation_words, characters)
    test_dataset = CharacterDataset(test_words, characters)

    return train_dataset, validation_dataset, test_dataset

In [5]:
def create_data_loader(dataset, window):
    x_list = []
    y_list = []
    for i, word in enumerate(dataset):
        for j, _ in enumerate(word):
            if j + window > len(word) - 1:
                break
            word_window = word[j:j + window]
            x, y = word_window[:-1], word_window[-1]
            x_list.append(x)
            y_list.append(y)

    # Create a DataLoader with the input-output pairs
    return DataLoader(
        TensorDataset(torch.tensor(x_list), torch.tensor(y_list)),
        batch_size,
        shuffle=True
    )

In [6]:
class NGramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, ngram_level):
        super(NGramLanguageModel, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.ngram_level = ngram_level

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_layer = nn.Parameter(torch.zeros((ngram_level - 1) * embedding_dim, hidden_dim))
        self.output_layer_W = nn.Parameter(torch.zeros((ngram_level - 1) * embedding_dim, vocab_size))
        self.output_layer_U = nn.Parameter(torch.zeros(hidden_dim, vocab_size))

        self.bias_output = torch.nn.Parameter(torch.ones(vocab_size))
        self.bias_hidden = torch.nn.Parameter(torch.ones(hidden_dim))

        self.init_weights()

    def init_weights(self):
        initrange = 0.5

    def forward(self, x):
        # Get character embeddings
        x = self.embedding(x)

        # Get the batch size
        batch_size = x.shape[0]

        # Reshape the input to a 2D tensor
        x = x.view(batch_size, -1)

        # Compute the output using the hidden and output layer parameters
        y = self.bias_output + torch.matmul(x, self.output_layer_W) + torch.matmul(torch.tanh(self.bias_hidden + torch.matmul(x, self.hidden_layer)), self.output_layer_U)

        return y

In [7]:
def compute_perplexity(total_loss, total_batches):
    # Compute the perplexity given the total loss and number of batches
    return torch.exp(torch.tensor(total_loss / total_batches)).item()

In [8]:
#Set up model
def train_model(data_loader, model, optimizer, criterion, epoch):
    model.train()
    total_loss, total_batches = 0.0, 0.0
    log_interval = 500

    for idx, (x, y) in tqdm(enumerate(data_loader)):
        optimizer.zero_grad()

        if not x.nelement():
            continue

        # Forward pass
        logits = model(x)

        # Compute the loss
        loss = criterion(input=logits, target=y.squeeze(-1))
        # Backward pass
        loss.backward()

        # Clip gradients to avoid exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # Update the model parameters
        optimizer.step()
        total_loss += loss.item()
        total_batches += 1

        # Log the training progress
        if idx % log_interval == 0 and idx > 0:
            perplexity = compute_perplexity(total_loss, total_batches)
            print(
                "| epoch {:3d} "
                "| {:5d}/{:5d} batches "
                "| perplexity {:8.3f} "
                "| loss {:8.3f} "
                .format(
                    epoch,
                    idx,
                    len(data_loader),
                    perplexity,
                    total_loss / total_batches,
                )
            )
            total_loss, total_batches = 0.0, 0

In [9]:
def evaluate_model(data_loader, model, criterion):
    model.eval()
    total_loss, total_batches = 0.0, 0

    with torch.no_grad():
        for idx, (x, y) in enumerate(data_loader):
            # Forward pass
            logits = model(x)
            # Compute the loss
            total_loss += criterion(input=logits, target=y.squeeze(-1)).item()
            total_batches += 1
    return total_loss / total_batches, compute_perplexity(total_loss, total_batches)

In [10]:
def generate_word(model, dataset, window=ngram_level):
    generated_word = []
    context = [0] * (ngram_level - 1)

    while True:
        # Get the logits for the current context
        logits = model(torch.tensor(context).view(1, -1))

        # Convert logits to probabilities using softmax
        probs = nn.Softmax(dim=1)(logits)

        # Sample a character index based on the probabilities
        token_id = torch.multinomial(probs, num_samples=1).item()

        generated_word.append(token_id)

        # Update the context with the generated character
        context = context[1:] + [token_id]

        # Stop generating if the end marker is encountered
        if token_id == 0:
            break

    # Decode the generated character indices to a word
    return ''.join(dataset.decode(generated_word))

In [11]:
#Load Data
train_dataset, validation_dataset, test_dataset = load_datasets(ngram_level)

# Create data loaders
train_loader = create_data_loader(train_dataset, ngram_level)
validation_loader = create_data_loader(validation_dataset, ngram_level)
test_loader = create_data_loader(test_dataset, ngram_level)

The number of examples in the dataset: 32033
The number of unique characters in the vocabulary: 27
The vocabulary we have is: .abcdefghijklmnopqrstuvwxyz
We've split up the dataset into 30033, 500, 1500 training, validation, and test examples


In [12]:
# Set up the model, loss function, and optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
model = NGramLanguageModel(
    train_dataset.get_vocab_size(), embedding_dim, hidden_dim, ngram_level
).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)


In [13]:
# Train the model
for epoch in range(1, num_epochs + 1):
    epoch_start_time = time.time()
    train_model(train_loader, model, optimizer, criterion, epoch)
    loss_val, perplexity_val = evaluate_model(validation_loader, model, criterion)
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} "
        "| time: {:5.2f}s "
        "| valid perplexity {:8.3f} "
        "| valid loss {:8.3f}".format(
            epoch,
            time.time() - epoch_start_time,
            perplexity_val,
            loss_val
        )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
loss_test, perplexity_test = evaluate_model(test_loader, model, criterion)
print("test perplexity {:8.3f} | test loss {:8.3f} ".format(perplexity_test, loss_test))


663it [00:00, 951.43it/s]

| epoch   1 |   500/15242 batches | perplexity    9.989 | loss    2.301 


1169it [00:01, 1215.90it/s]

| epoch   1 |  1000/15242 batches | perplexity    8.874 | loss    2.183 


1698it [00:01, 1274.74it/s]

...

| epoch   5 | 14500/15242 batches | perplexity    7.379 | loss    1.999 


15101it [00:14, 966.35it/s]

| epoch   5 | 15000/15242 batches | perplexity    7.213 | loss    1.976 


15242it [00:14, 1044.87it/s]


-----------------------------------------------------------
| end of epoch   5 | time: 14.64s | valid perplexity    7.567 | valid loss    2.024
-----------------------------------------------------------
Checking the results of test dataset.
test perplexity    7.400 | test loss    2.001 


In [14]:
# Generate some words with model
for _ in range(50):
    print(generate_word(model, train_dataset))

brconi.
morijush.
orisa.
bellitylie.
digh.
praylee.
ley.
chril.
ceiblayvennse.
jadi.
nev.
lastapraeno.
corsen.
lavi.
simanttyn.
stre.
ngeri.
rah.
ern.
adrey.
mangeo.
for.
kharo.
barol.
kachanaca.
ren.
kayne.
lee.
ania.
mai.
bondy.
laver.
ayne.
shi.
myani.
shimo.
bran.
keid.
moristovisspperisaree.
mer.
sumbritonta.
eavay.
shilonn.
sodel.
maestoa.
rayadisiny.
rann.
tik.
gona.
faydhele.
